[Reference](https://medium.com/@johnidouglasmarangon/python-useful-code-snippets-864990b8e900)

# Class-Level Properties and Initialization Methods

In [1]:
from typing import Optional


class MyAppProperties:
    timeout: int
    db_uri: str
    prefix: Optional[str] = None

    @classmethod
    def init(
        cls,
        timeout: int,
        db_uri: str,
        prefix: str = "my-app",
    ) -> None:
        cls.timeout = timeout
        cls.db_uri = db_uri
        cls.prefix = prefix

In [2]:
MyAppProperties.init(
    timeout=10,
    db_uri="sqlite:///database.db",
)


print(MyAppProperties.timeout)
print(MyAppProperties.db_uri)

10
sqlite:///database.db


# Timeout Middleware in FastAPI to Handle Long-Running Requests

In [4]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00


In [6]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 1.8 MB/s eta 0:00:00


In [8]:
from fastapi import FastAPI, Request, Response, status
from fastapi.responses import PlainTextResponse
import asyncio
from typing import Callable


app = FastAPI()


@app.middleware("http")
async def timeout_middleware(
    request: Request,
    call_next: Callable,
) -> Response:
    try:
        timeout = float(request.headers.get("X-Timeout-Request", 10.0)) # 10.0 is a default value

        return await asyncio.wait_for(
            call_next(request),
            timeout=timeout,
        )
    except asyncio.TimeoutError:
        return PlainTextResponse(
            status_code=status.HTTP_408_REQUEST_TIMEOUT,
            content="Request timed out",
        )


@app.get("/")
async def root(delay: int = 5):
    await asyncio.sleep(delay)
    return {"message": f"Waited for {delay} seconds"}


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

```
curl -v -XGET 'http://0.0.0.0:8000?delay=7' \
    -H 'X-Timeout-Request: 5.0' \
    -H 'Content-type: application/json'
```

# Custom Timeout Decorator


In [9]:
import signal
import functools

class TimeoutError(Exception):
    pass

def timeout(seconds):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(f"Timeout on '{func.__name__}'")

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Set the signal handler and a timeout alarm
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                # Disable the alarm after the function completes
                signal.alarm(0)
            return result

        return wrapper

    return decorator

In [10]:
@timeout(5)
def long_running_task():
    # Simulate a long task
    time.sleep(10) # Will raise a timeout exception
    return "Task completed"

# Leveraging SQLite with Threads in Python

In [11]:
import sqlite3
import threading


def connection():
    conn = sqlite3.connect("file::memory:?cache=shared", isolation_level=None)
    return conn


def insert(name):
    with connection() as conn:
        conn.execute("INSERT INTO users (name) VALUES (?);", (name,))


def main():
    with connection() as conn:
        conn.execute("CREATE TABLE users (name TEXT NOT NULL)")

    threads = []
    for i in range(1000):
        t = threading.Thread(target=insert, args=(f"{str(i + 1)}",))
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    with connection() as conn:
        row = conn.execute("SELECT count(*) FROM users;").fetchone()
        print(row)


if __name__ == "__main__":
    main()

(1000,)
